In [1]:
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, SeparableConv2D
from keras.layers import Bidirectional, RepeatVector, TimeDistributed, Permute, Embedding
from keras.layers import concatenate, add, Lambda, multiply
from keras.layers import LSTM
from keras.models import Model
from keras.regularizers import l2
from keras.preprocessing.sequence import pad_sequences

from keras.applications import imagenet_utils, Xception
from keras.preprocessing.image import img_to_array
import keras.backend as K

import numpy as np 
import json
import os
import cv2
import h5py
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
from helpers import TrainingMonitor
from helpers import Utils

In [4]:
X_file = open('../input/dataset/X.pickle', 'rb')
x = pickle.load(X_file)
X_file.close()

In [5]:
Y_file = open('../input/dataset/Y.pickle', 'rb')
y = pickle.load(Y_file)
Y_file.close()

In [6]:
y = np.array(y)

In [7]:
y_input = np.zeros([1000, 10])
y_input[:, 1:] = y[:,:-1]
y_input[:, 0] = -1

In [8]:
y_input[0]

array([ -1.,   2.,  10.,  10.,   9.,   0.,   1.,   6.,   0.,   1.])

In [9]:
Yoh_file = open('../input/dataset/Yoh.pickle', 'rb')
yoh = pickle.load(Yoh_file)
Yoh_file.close()

In [10]:
train_split, val_split = 0.7, 0.15

train_len = int(train_split * x.shape[0])
train_val_len = int((train_split + val_split) * x.shape[0])

x_train = x[:train_len]
x_val = x[train_len:train_val_len]
x_test = x[train_val_len:]

y_input_train = y_input[:train_len]
y_input_val = y_input[train_len:train_val_len]
y_input_test = y_input[train_val_len:]

yoh_train = yoh[:train_len]
yoh_val = yoh[train_len:train_val_len]
yoh_test = yoh[train_val_len:]

In [11]:
x_train.shape

(700, 30)

In [12]:
y_input_train.shape

(700, 10)

In [13]:
yoh_train.shape

(700, 10, 11)

In [14]:
INPUT_LENGTH = 30
OUTPUT_LENGTH = 10
input_dict_size = 37
output_dict_size = 11

In [15]:
encoder_input = Input(shape=(INPUT_LENGTH,))
decoder_input = Input(shape=(OUTPUT_LENGTH,))

In [16]:
encoder = Embedding(input_dict_size, 64, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
encoder = LSTM(64)(encoder)
print(encoder.get_shape())

(?, 64)


In [17]:
decoder = Embedding(output_dict_size, 64, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)

decoder = LSTM(64, return_sequences=True)(decoder, initial_state=[encoder, encoder])
decoder = TimeDistributed(Dense(output_dict_size, activation="softmax"))(decoder)

print(decoder.get_shape())

(?, 10, 11)


In [18]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[decoder])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["accuracy"])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 64)       2368        input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 10, 64)       704         input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [19]:
history = model.fit(x=[x_train, y_input_train], y=[yoh_train],
          validation_data=([x_val, y_input_val], [yoh_val]),
          verbose=1,
          batch_size=64,
          epochs=50)

Train on 700 samples, validate on 150 samples
Epoch 1/50
700/700 [==============================] - 3s 5ms/step - loss: 0.3021 - acc: 0.9091 - val_loss: 0.2973 - val_acc: 0.9091
Epoch 2/50
700/700 [==============================] - 1s 2ms/step - loss: 0.2865 - acc: 0.9091 - val_loss: 0.2759 - val_acc: 0.9091
Epoch 3/50
700/700 [==============================] - 1s 2ms/step - loss: 0.2699 - acc: 0.9091 - val_loss: 0.2627 - val_acc: 0.9091
Epoch 4/50
700/700 [==============================] - 1s 2ms/step - loss: 0.2580 - acc: 0.9091 - val_loss: 0.2513 - val_acc: 0.9089
Epoch 5/50
700/700 [==============================] - 1s 2ms/step - loss: 0.2469 - acc: 0.9114 - val_loss: 0.2397 - val_acc: 0.9129
Epoch 6/50
700/700 [==============================] - 1s 2ms/step - loss: 0.2355 - acc: 0.9157 - val_loss: 0.2281 - val_acc: 0.9264
Epoch 7/50
700/700 [==============================] - 1s 2ms/step - loss: 0.2240 - acc: 0.9266 - val_loss: 0.2164 - val_acc: 0.9292
Epoch 8/50
700/700 [==========

In [20]:
scores = model.evaluate([x_test, y_input_test], yoh_test, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )

Train: 94.73%; Val: 94.77%; Test: 94.82%


In [ ]:
output_path = "../output/"

print(os.getpid())
filepath=output_path + "progress/seq2seq-stacked-weights-best.hdf5"
MC = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

figPath = os.path.sep.join([output_path, "monitor/{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([output_path, "monitor/{}.json".format(os.getpid())])
TM = TrainingMonitor(figPath, jsonPath=jsonPath, startAt=0)

RLR = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

callbacks = [MC, TM, RLR]

In [21]:
model_json = model.to_json()
with open("../output/saved/date_model_seq2seq_94.33.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("../output/saved/date_weight_seq2seq_94.33.hdf5")

C:\Users\boringtao\Anaconda3\lib\site-packages\keras\engine\topology.py:2361: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/TensorArrayReadV3:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_1/TensorArrayReadV3:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '
